In [1]:
# import module
import requests
from bs4 import BeautifulSoup
from pathlib import Path

In [2]:
asin = 'B00J5P7GIA'
url = 'https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?sortBy=recent&pageNumber={page}'

In [3]:
HEADERS = ({'User-Agent':
			'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
			AppleWebKit/537.36 (KHTML, like Gecko) \
			Chrome/90.0.4430.212 Safari/537.36',
			'Accept-Language': 'en-US, en;q=0.5'})

# user define function
# Scrape the data
def getdata(url):
	r = requests.get(url, headers=HEADERS)
	if r.status_code == 200:
		return r.text
	else:
		print(r)


def html_code(url):

	# pass the url
	# into getdata function
	htmldata = getdata(url)
	soup = BeautifulSoup(htmldata, 'html.parser')

	# display html code
	return (soup)

def extract_reviews(soup):
	reviews = []
	outer_spans = soup.find_all('span', attrs={'data-hook': 'review-body'})
	for outer_span in outer_spans:
		inner_span = outer_span.find('span')
		review_text = inner_span.text.strip()
		reviews.append(review_text)

	return reviews


In [4]:
for page in [1,2,3,4]:
    print(        'Scraping page {page} of reviews for {asin}'.format(page=page, asin=asin)
    )
    soup = html_code(url.format(asin=asin, page=page))
    reviews = extract_reviews(soup)
    with Path('raw_reviews/{asin}_{page}.txt'.format(asin=asin, page=page)).open('w') as f:
        f.write('\n'.join(reviews))


Scraping page 1 of reviews for B00J5P7GIA
Scraping page 2 of reviews for B00J5P7GIA
Scraping page 3 of reviews for B00J5P7GIA
Scraping page 4 of reviews for B00J5P7GIA


In [5]:
url.format(asin=asin, page=page)

'https://www.amazon.com/product-reviews/B00J5P7GIA/ref=cm_cr_arp_d_viewopt_srt?sortBy=recent&pageNumber=4'